In [15]:
import pandas as pd
import os
import numpy as np

In [16]:
X = pd.read_csv('/content/drive/MyDrive/training_set_rel3.csv', sep='\t',encoding='ISO-8859-1')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [17]:
y = X['domain1_score']
print(y)
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

0         8
1         9
2         7
3        10
4         8
         ..
12971    35
12972    32
12973    40
12974    40
12975    40
Name: domain1_score, Length: 12976, dtype: int64


In [18]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [19]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

In [20]:
import tensorflow_hub as hub

use = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")
aa=[]

In [22]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from keras.layers import Bidirectional

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(Bidirectional(LSTM(300, return_sequences=True),input_shape=[1, 12288]))
    #model.add(Bidirectional(LSTM(300, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True)))
    model.add(Bidirectional(LSTM(64, recurrent_dropout=0.4)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])
    model.summary()

    return model

In [23]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import sent_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    #tokennizer=PunktSentenceTokenizer(english.pickle)
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append((raw_sentence))
    #print(sentences)
    return sentences

def makeFeatureVec(words, num_features,n):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((n,num_features),dtype="float32")
    #success
    x=use(words) #length of feature vector is (16,512) and length of use is (16,128)
    y=len(x[0])
#     featureVec = np.zeros((n,num_features),dtype="float32")
    featureVec = np.zeros((n,y),dtype="float32")
    featureVec = np.add(featureVec,x).flatten() # getting error while adding vectors
    ss=num_features-y
#     print("diff=",ss)
    featureVec = np.pad(featureVec, (0,ss), 'constant', constant_values=(0, 0))
#     print("AFTER PADDING",featureVec,len(featureVec))
#     print("SHAPE",featureVec.shape)     
    #featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, num_features,aa):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features*96),dtype="float32") # creating max length feature vector having 0s
#     print(num_features*96)
    for essay in essays:
        
#         #essayFeatureVecs.insert(counter,makeFeatureVec(essay,num_features,aa[counter]))
        arr=makeFeatureVec(essay, num_features,aa[counter]).flatten() # getting error here
#         print(arr)
        ss=(num_features*96)-len(arr)
        arr1=np.pad(arr,(0,ss),'constant', constant_values=(0, 0))
#         #print(np.add(arr1,np.zeros(num_features*np.max(aa))))
        essayFeatureVecs[counter] = np.add(arr1,np.zeros((num_features*96),dtype="float"))
#         #makeFeatureVec(essay, num_features,aa[counter]).flatten()
#         #print(essayFeatureVecs[0])
        counter = counter + 1
    return essayFeatureVecs

In [ ]:
#TESTING CELL

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
import tensorflow as tf
l1=[]
l2=[]
y_train=y[0]
clean_train_essays=[]
clean_test_essays=[]
train_essay=X.iloc[0,2]
test_essay=X.iloc[100,2]
num_features = 512
sentences=[]
sentences += essay_to_sentences(train_essay, remove_stopwords = False) # converting paragraph to sentences
print("done 1")

l1.append(len(sent_tokenize(train_essay))) # adding length of sentences to l1
l2.append(len(sent_tokenize(test_essay)))
print("done 2")
clean_train_essays.append(essay_to_sentences(train_essay, remove_stopwords=True))
clean_test_essays.append(essay_to_sentences(test_essay, remove_stopwords=True))
print("done 3")
a=np.array(l1)
b=np.array(l2)
print("done 4")
trainDataVecs = getAvgFeatureVecs(clean_train_essays, num_features,a)
testDataVecs = getAvgFeatureVecs(clean_test_essays, num_features,b)
print("done 5")
trainDataVecs=np.array(trainDataVecs)
testDataVecs=np.array(testDataVecs)
print("done 6")
trainDataVecs=np.reshape(trainDataVecs,(trainDataVecs.shape[0],1,trainDataVecs.shape[1]))
testDataVecs=np.reshape(testDataVecs,(testDataVecs.shape[0],1,testDataVecs.shape[1]))
print("done 7")
lstm_model = get_model()
history=lstm_model.fit(trainDataVecs, y_train, batch_size=1, epochs=5)
print("done 8")
y_pred=(lstm_model.predict(testDataVecs))
print("done 9")
# if count == 6:
#       break
         #lstm_model.save('./model_weights/final_lstm.h5')
    
    # Round y_pred to the nearest integer.
y_pred = np.around(y_pred)
print(y_pred)
print("done 10")
#     # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result))
print("done 11")
# results.append(result)
    
 




In [25]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
import tensorflow as tf

cv = KFold(n_splits = 5, shuffle = True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    #print(y_train)
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    y_train=y_train
    y_test=y_test
    num_features = 128
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3
    clean_train_essays = []
    clean_test_essays=[]
    #essays=train_essays
    sentences=[]
    l1=[]
    l2=[]
    for essay in train_essays:
      sentences += essay_to_sentences(essay, remove_stopwords = False)
  
      #print(sentences)
    for essay in test_essays:
      sentences += essay_to_sentences(essay, remove_stopwords = False)

    for essay_v in train_essays:
      l1.append(len(sent_tokenize(essay_v)))
      clean_train_essays.append(essay_to_sentences(essay_v, remove_stopwords=True))
      #print(clean_train_essays)
    a=np.array(l1)
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, num_features,a)
    for essay_v in test_essays:
      l2.append(len(sent_tokenize(essay_v)))
      clean_test_essays.append(essay_to_sentences(essay_v, remove_stopwords=True))
    b=np.array(l2)
    testDataVecs=getAvgFeatureVecs(clean_test_essays, num_features,b)
    trainDataVecs=np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    trainDataVecs=np.reshape(trainDataVecs,(trainDataVecs.shape[0],1,trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
#     print(b)
    lstm_model = get_model()
    history=lstm_model.fit(trainDataVecs, y_train, batch_size=60, epochs=5)
    y_pred=(lstm_model.predict(testDataVecs))
    #print(y_pred)
    # Save any one of the 5 models.
    if count == 6:
      break
         #lstm_model.save('./model_weights/final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
#     print(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)
    
    count += 1
   
    


--------Fold 1--------

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_10 (Bidirectio (None, 1, 600)            30213600  
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 128)               340480    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 30,554,209
Trainable params: 30,554,209
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
173/173 [==============================] - 76s 368ms/step - loss: 47.9664 - accuracy: 0.1160
Epoch 2/5
173/173 [==============================] - 61s 354ms/step - loss: 14.4313 - accuracy: 0.1286
Epoch 3/5

In [26]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.9575


In [ ]:
arr = np.array([1, 3, 2, 5, 4])
arr1=np.zeros(5)
print(np.add(arr,arr1))
  
# padding array using CONSTANT mode
pad_arr = np.pad(arr, (0, 8), 'constant', constant_values=(0, 0))
  
print(pad_arr)